In [79]:
import os
import pandas as pd
import math
#Change working directory
folder = 'C:/Users/LENOVO/Desktop/data/A5 176'
os.chdir(folder)

#To approach files rapidly
def walk_through_files(TARGET, ext=""):
    import os
    lst = []
    for root, dirs, files in os.walk(TARGET):
        for file in files:
            if ext == "" or file.endswith(ext):
                lst.append(os.path.join(root, file))
    return lst

data = walk_through_files(folder, ext='xlsx')
VN30 = pd.DataFrame(pd.read_excel('C:/Users/LENOVO/Desktop/data/VN30_weights.xlsx', header = 0))
cafef = pd.DataFrame(pd.read_excel('C:/Users/LENOVO/Desktop/data/Pricecafef.xlsx', header = 0))

#Data pre-processing
SSIraw = pd.DataFrame(pd.read_excel(data[5], header = 7)) #Leave 7 starting rows
SSIraw = SSIraw.rename(columns={
            'Account': 'id',
            'Name': 'name',
            'ID': 'account',
            'Address': 'address',
            'B_MVol': 'buy',
            'S_MVol': 'sell',
            'B_OVol': 'buyOrder',
            'S_OVol': 'sellOrder',
            #'Buy Value List': 'buyvalue',
            #'Buy Numlots': 'buynumlot',
            #'Code': 'stock',
            #'Day': 'day',
            #'Price': 'price',
            })
SSI = SSIraw.groupby(['id','name', 'account', 'address']).sum()
code_SSI = ['SSI']*(len(SSI))
SSI['Code'] = code_SSI
SSI['Day'] = [20220617]*(len(SSI))
price_SSI = [float(cafef[cafef['stock'] == 'SSI']['avg'])]*(len(SSI))
SSI['Price'] = price_SSI
#SSI['Buy Value List'] = SSI['B_MVol']*int((VN30[VN30['stock'] == 'SSI']['refPrice']))
#SSI['Buy Numlots'] = SSI['Buy Value List'] / float((VN30[VN30['stock'] == 'SSI']['value']))
#SSI['Buy Numlots'] = ["{:.2f}".format(value) for value in SSI['Buy Numlots']]

SSI.drop(['%B_Ovol', '%B_MVol','%S_Ovol', '%S_MVol'], axis=1, inplace=True)
SSI = SSI.rename(columns={
            #'Account': 'id',
            #'Name': 'name',
            #'ID': 'account',
            #'Address': 'address',
            #'B_MVol': 'buy',
            #'S_MVol': 'sell',
            #'B_OVol': 'buyOrder',
            #'S_OVol': 'sellOrder',
            #'Buy Value List': 'buyvalue',
            #'Buy Numlots': 'buynumlot',
            'Code': 'stock',
            'Day': 'day',
            'Price': 'price',
            })
dfB  = SSI.sort_values(by=['buy'])
dfS  = SSI.sort_values(by=['sell'])

SSI.to_excel(f'zSSI_processed.xlsx')
SSI = pd.DataFrame(pd.read_excel('zSSI_processed.xlsx', header = 0))
#SSI2 = pd.DataFrame(pd.read_excel('zSSI_processed.xlsx', header = 0))
dfB2  = SSI.nlargest(20, 'buy')
#dfB2 = dfB.tail(10)
dfS2 = SSI.nlargest(20, 'sell')
#dfS2 = dfS.tail(10)


VN30 = pd.DataFrame(pd.read_excel('C:/Users/LENOVO/Desktop/data/VN30_weights.xlsx', header = 0))
ref = int((VN30[VN30['stock'] == 'SSI']['refPrice']))

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
top20buy_name = dfB2['name']
top20buy_id = dfB2['id']
top20buy_volume = dfB2['buy']
top20sell_name = dfS2['name']
top20sell_id = dfS2['id']
top20sell_volume = dfS2['sell']
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
writer = pd.ExcelWriter('zSummary3.xlsx', engine='xlsxwriter')
workbook = writer.book
worksheet1 = workbook.add_worksheet()

worksheet1.write_row('A5', ['name', 'id', 'buy'], workbook.add_format({'bold': 1}))
worksheet1.write_column('A6', top20buy_name)
worksheet1.write_column('B6', top20buy_id)
worksheet1.write_column('C6', top20buy_volume)
worksheet1.write_row('E5', ['name', 'id', 'sell'], workbook.add_format({'bold': 1}))
worksheet1.write_column('E6', top20sell_name)
worksheet1.write_column('F6', top20sell_id)
worksheet1.write_column('G6', top20sell_volume)

worksheet1.write_row('A1', ['Stock:', 'SSI'], workbook.add_format({'bold': 1}))
worksheet1.write_row('A2', ['Day:', '17-06-2022'], workbook.add_format({'bold': 1}))
worksheet1.write_row('A4', ['Top 20 buyers'], workbook.add_format({'bold': 1}))
worksheet1.write_row('A4', ['Top 20 sellers'], workbook.add_format({'bold': 1}))

numberoftraders = len(SSI)
numberoftrades = (SSI['buy'] >0).sum() + (SSI['sell'] >0).sum()

worksheet1.write_row('D1', ['Num_trades:', numberoftrades], workbook.add_format({'bold': 1}))
worksheet1.write_row('D2', ['Num_traders:', numberoftraders], workbook.add_format({'bold': 1}))
worksheet1.write_row('D3', ['Num_buyers:', int((SSI['buyOrder'] >0).sum())], workbook.add_format({'bold': 1}))
worksheet1.write_row('D4', ['Num_seller', int((SSI['sellOrder'] >0).sum())], workbook.add_format({'bold': 1}))


worksheet1.write_row('F1', ['Total buy volume:', int(SSI['buy'].sum())], workbook.add_format({'bold': 1}))
worksheet1.write_row('F2', ['Total buy value', int(int(SSI['buy'].sum())*ref)], workbook.add_format({'bold': 1}))
worksheet1.write_row('F3', ['Total sell volume:', int(SSI['sell'].sum())], workbook.add_format({'bold': 1}))
worksheet1.write_row('F4', ['Total sell value:', int(int(SSI['buy'].sum())*ref)], workbook.add_format({'bold': 1}))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#Count number of buyers
#dfB  = SSI.sort_values(by=['B_MVol'])
worksheet2 = workbook.add_worksheet()

i = 0
while dfB['buy'][i] == 0:
    i = i+1
minvol = dfB['buy'][i]   
interval = (SSI['buy'].max() - minvol)/1000
xcoor = [minvol]
for k in range(1,1002):
    xcoor.append(k*interval)
ycoor = []
def count(list, l, r):
    c = 0
    for x in list:
        # condition check
        if x> l and x<= r:
            c = c + 1
    return c
for l in range(1001):
    ycoor.append(count(SSI['buy'], xcoor[l], xcoor[l+1]))
#df.to_excel(f'SSIclean.xlsx')
#df2 = pd.DataFrame()
xcoor2 = []
ycoor2 = []
for m in range(1001):
    if ycoor[m] > 0:
        xcoor2.append(int(xcoor[m]))
        ycoor2.append(int(ycoor[m]))
xcoor3 = [str(xcoor2[0]) + '-' + str(int(xcoor2[1]))]
for n in range(1, len(xcoor2)):
    xcoor3.append(str(xcoor2[n]) + '-' + str(int(xcoor2[n] + interval)))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#Count number of sellers
#dfS  = SSI.sort_values(by=['S_MVol'])
i = 0
while dfS['sell'][i] == 0:
    i = i+1
minvol = dfS['sell'][i]   
interval = (SSI['sell'].max() - minvol)/1000
zcoor = [minvol]
for k in range(1,1002):
    zcoor.append(k*interval)
tcoor = []
def count(list, l, r):
    c = 0
    for x in list:
        # condition check
        if x> l and x<= r:
            c = c + 1
    return c
for l in range(1001):
    tcoor.append(count(SSI['sell'], zcoor[l], zcoor[l+1]))
zcoor2 = []
tcoor2 = []
for m in range(1001):
    if ycoor[m] > 0:
        zcoor2.append(int(zcoor[m]))
        tcoor2.append(int(tcoor[m]))
zcoor3 = [str(zcoor2[0]) + '-' + str(int(zcoor2[1]))]
for n in range(1, len(zcoor2)):
    zcoor3.append(str(zcoor2[n]) + '-' + str(int(zcoor2[n] + interval)))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#Classifying Sharks, Wolves, Sheep
"""
shark = 0
wolf = 0
sheep = 0
# df.iloc
# df.loc
for w in range(len(dfB['buy'])):
    if dfB['buy'][w]*ref < 5*10^8:
        sheep = sheep + 1
for w in range(len(dfB['buy'])):
    if 5*10^8 <= dfB['buy'][w]*ref < 10^12:
        wolf = wolf + 1
shark = len(dfB['buy']) - wolf - sheep
sws = ['Sharks', 'Wolves', 'Sheep']
swsvalue = [shark, wolf, sheep]
"""
# dfB['buyValue'] = dfB['buy'] * ref
# dfB[(dfB['buy'] * ref >= 500_000_000) & 
#     (dfB['buy'] * ref <= 1000_000_000_000)].reset_index()[['Name', 'buy', 'buyValue']]
sheep = (dfB['buy'] * ref <= 200_000_000).sum()
wolf = ((dfB['buy'] * ref >= 200_000_000) & (dfB['buy'] * ref <= 1_000_000_000)).sum()
shark = len(dfB['buy']) - wolf - sheep
sws = ['Sharks', 'Wolves', 'Sheep']
swsvalue = [shark, wolf, sheep]

# Add the worksheet data that the charts will refer to
headings = ['Buy volume', 'Num_trade', 'Sell volume', 'Num_trade', 'Type', 'Num_investors']
worksheet2.write_row('A1', headings, workbook.add_format({'bold': 1}))
worksheet2.write_column('A2', xcoor3)
worksheet2.write_column('B2', ycoor2)
worksheet2.write_column('C2', zcoor3)
worksheet2.write_column('D2', tcoor2)
worksheet2.write_column('E2', sws)
worksheet2.write_column('F2', swsvalue)


# Create a new Chart object
chart1 = workbook.add_chart({'type': 'column'})

#     [sheetname, first_row, first_col, last_row, last_col]
chart1.add_series({
    'categories': ['Sheet2', 1, 0, len(xcoor3)+1, 0],
    'values':     ['Sheet2', 1, 1, len(ycoor2)+1, 1],
    'line':       {'color': 'black'},
    'gap': 0
    })


# Add a chart title and some axis labels.
chart1.set_title ({'name': 'Distribution Buy Matching Volume'})
chart1.set_x_axis({'name': 'Volume', 'num_font':  {'size': 7, 'rotation': 90}})
chart1.set_y_axis({'name': 'Number of investors'})

chart1.set_size({'x_scale': 1, 'y_scale': 1})
# Set an Excel chart style.
chart1.set_style(10)

# Insert the chart into the worksheet (with an offset).
worksheet1.insert_chart('A28', chart1)

###########################################################
# Create a new Chart object.
chart2 = workbook.add_chart({'type': 'column'})

#     [sheetname, first_row, first_col, last_row, last_col]
chart2.add_series({
    'categories': ['Sheet2', 1, 2, len(zcoor3)+1, 2],
    'values':     ['Sheet2', 1, 3, len(tcoor2)+1, 3],
    'line':       {'color': 'black'},
    'gap': 0
    })


# Add a chart title and some axis labels.
chart2.set_title ({'name': 'Distribution Sell Matching Volume'})
chart2.set_x_axis({'name': 'Volume', 'num_font':  {'size': 7, 'rotation': 90}})
chart2.set_y_axis({'name': 'Number of investors'})

chart2.set_size({'x_scale': 1, 'y_scale': 1})
# Set an Excel chart style.
chart2.set_style(5)
#chart2.set_rotation(90)

# Insert the chart into the worksheet (with an offset).
worksheet1.insert_chart('I28', chart2)

###########################################################
# Create a new Chart object.
chart3 = workbook.add_chart({'type': 'pie'})

#     [sheetname, first_row, first_col, last_row, last_col]
chart3.add_series({
    'categories': ['Sheet2', 1, 4, 3, 4],
    'values':     ['Sheet2', 1, 5, 3, 5],
    'points': [
        {'fill': {'color': 'blue'}},
        {'fill': {'color': 'green'}},
        {'fill': {'color': 'red'}},
        ],
})


# Add a chart title and some axis labels.
chart3.set_title ({'name': 'Sharks Wolves Sheep'})
#chart3.set_x_axis({'name': 'Type of investors', 'num_font':  {'size': 7, 'rotation': 90}})
#chart3.set_y_axis({'name': 'Number of investors'})

chart3.set_size({'x_scale': 1, 'y_scale': 1})
# Set an Excel chart style.
chart3.set_style(8)

# Insert the chart into the worksheet (with an offset).
#worksheet.insert_chart('G15', chart2)
worksheet1.insert_chart('A44', chart3)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
SSI['net'] = SSI['buy'] - SSI['sell']
dfnet = SSI.nlargest(int(len(SSI)), 'net')
dfnet.to_excel(f'zSSI_ranked.xlsx')
interval2 = int(len(SSI))/100
dfnet2 = pd.DataFrame(pd.read_excel('zSSI_ranked.xlsx', header = 0))
ranking = list(range(100))
netvolume = []
for k in range(100):
    netvolume.append(dfnet2['net'][abs(math.floor(interval2*k)): abs(math.floor(interval2*(k+1)))].sum())
        

worksheet2.write_column('G2', ranking)
worksheet2.write_column('H2', netvolume)
###########################################################
# Create a new Chart object.
chart4 = workbook.add_chart({'type': 'line'})

#     [sheetname, first_row, first_col, last_row, last_col]
chart4.add_series({
    'categories': ['Sheet2', 1, 6, len(ranking)+1, 6],
    'values':     ['Sheet2', 1, 7, len(ranking)+1, 7],
    'points': [
        {'fill': {'color': 'blue'}},
        {'fill': {'color': 'green'}},
        {'fill': {'color': 'red'}},
        ],
})


# Add a chart title and some axis labels.
chart4.set_title ({'name': 'Line chart based on net volume'})
chart4.set_x_axis({'name': 'Rankings', 'num_font':  {'size': 7, 'rotation': 90}})
chart4.set_y_axis({'name': 'Netvolume', 'num_font':  {'size': 7}})

chart4.set_size({'x_scale': 1, 'y_scale': 1})
# Set an Excel chart style.
chart4.set_style(8)

# Insert the chart into the worksheet (with an offset).
#worksheet.insert_chart('G15', chart2)
worksheet1.insert_chart('I44', chart4)

workbook.close()

In [73]:
print(len(netvolume))
    
    
    
#df_netrank = SSI['net'].rank(method='max')

9735


In [76]:
dfnet2['net'][abs(math.floor(interval2*k)): abs(math.floor(interval2*(k+1)))].sum()

-10818200